##### 자료출처 : https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html#

torch geometric 설치

In [1]:
import torch

try:
    import torch_geometric
except ModuleNotFoundError:
    # Installing torch geometric packages with specific CUDA+PyTorch version.
    # See https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html for details
    TORCH = torch.__version__.split('+')[0]
    CUDA = 'cu' + torch.version.cuda.replace('.','')

    !pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-geometric

### 1. Data Handling of Graphs

In [2]:
import torch
import torch_geometric
from torch_geometric.data import Data

In [3]:
# 1. 그래프 만들기

# 노드 만들기 
x = torch.tensor([[-3],[0],[3],[5],[-2]], dtype=torch.float) # 순서대로 Node(0,1,2,..) 번호가 부여된다.          

# 엣지 만들기
edge_index = torch.tensor([[0,1,1,2,4],
                           [1,0,2,1,4]], dtype=torch.long) # 위 부여된 노드 번호대로 edge를 연결한다.
print("start node :",edge_index[0])
print("end node :",edge_index[1])

graph = Data(x=x, edge_index=edge_index)
print(graph,'\n')
print("※ x의 좌측값이 노드개수, edge_index의 우측값이 엣지개수 입니다. ※")

start node : tensor([0, 1, 1, 2, 4])
end node : tensor([1, 0, 2, 1, 4])
Data(x=[5, 1], edge_index=[2, 5]) 

※ x의 좌측값이 노드개수, edge_index의 우측값이 엣지개수 입니다. ※


In [4]:
# 2. 그래프 형태의 필수 조건을 갖췄는지 확인
# graph.validate(raise_on_error=True)

In [5]:
# 3. 그래프 속성 확인
print("그래프 KEY :",graph.keys)
print("노드 수 :",graph.num_nodes) # 노드 수 
print("엣지 수 :",graph.num_edges) # 엣지 수
print("노드 특징 행렬 :",graph.num_node_features) # 노드의 특징 행렬 ( -1, 0, 1 로 각 1개씩이라 1 리턴)
print("고립 노드 유무 :",graph.has_isolated_nodes()) # 고립된 노드 유무 --> (4번째노드는 연결된 edge가 없음)
print("셀프 루프 유무 :",graph.has_self_loops()) # self loop 존재 유무 --> (5번째노드는 self loop 존재)
print("그래프 방향성 여부 :",graph.is_directed()) # 그래프의 방향성 여부 

그래프 KEY : ['edge_index', 'x']
노드 수 : 5
엣지 수 : 5
노드 특징 행렬 : 1
고립 노드 유무 : True
셀프 루프 유무 : True
그래프 방향성 여부 : False


In [6]:
# 4. graph 객체를 cpu / gpu 변환 가능
device = torch.device('cuda')
graph = graph.to(device)

### 2. Common Benchmark Datasets

In [7]:
from torch_geometric.datasets import TUDataset

graph_dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')

# 그래프 속성 확인
print(f"전체 그래프 수 : {len(graph_dataset)}")
for idx,graph in enumerate(graph_dataset):
    print(f"====== {idx}번째 그래프의 특징은 다음과 같습니다. ======")
    print("그래프 KEY :",graph.keys)
    print("노드 수 :",graph.num_nodes) 
    print("엣지 수 :",graph.num_edges) 
    print("노드 특징 행렬 :",graph.num_node_features) 
    print("고립 노드 유무 :",graph.has_isolated_nodes())
    print("셀프 루프 유무 :",graph.has_self_loops()) 
    print("그래프 방향성 여부 :",graph.is_directed())
    break

# 데이터셋 추가 기능
# 1. 학습/검증/테스트 분류가능
train_dataset = graph_dataset[:540]
test_dataset = graph_dataset[540:]

# 2. 셔플가능
train_dataset = train_dataset.shuffle()

전체 그래프 수 : 600
====== 0번째 그래프의 특징은 다음과 같습니다. ======
그래프 KEY : ['edge_index', 'x', 'y']
노드 수 : 37
엣지 수 : 168
노드 특징 행렬 : 3
고립 노드 유무 : False
셀프 루프 유무 : False
그래프 방향성 여부 : False


In [8]:
from torch_geometric.datasets import Planetoid

graph_dataset = Planetoid(root='/tmp/Cora', name='Cora')

# 그래프 속성 확인
print(f"전체 그래프 수 : {len(graph_dataset)}")
for idx,graph in enumerate(graph_dataset):
    print(f"====== {idx}번째 그래프의 특징은 다음과 같습니다. ======")
    print("그래프 KEY :",graph.keys)
    print("노드 수 :",graph.num_nodes) 
    print("엣지 수 :",graph.num_edges) 
    print("노드 특징 행렬 :",graph.num_node_features) 
    print("고립 노드 유무 :",graph.has_isolated_nodes())
    print("셀프 루프 유무 :",graph.has_self_loops()) 
    print("그래프 방향성 여부 :",graph.is_directed())

    print("======= 추가된 특징 =======")
    print("학습되는 노드 수 :", graph.train_mask.sum())
    print("검증되는 노드 수 :", graph.val_mask.sum())
    print("테스트되는 노드 수 :", graph.test_mask.sum())
    break

전체 그래프 수 : 1
====== 0번째 그래프의 특징은 다음과 같습니다. ======
그래프 KEY : ['x', 'y', 'train_mask', 'test_mask', 'edge_index', 'val_mask']
노드 수 : 2708
엣지 수 : 10556
노드 특징 행렬 : 1433
고립 노드 유무 : False
셀프 루프 유무 : False
그래프 방향성 여부 : False
======= 추가된 특징 =======
학습되는 노드 수 : tensor(140)
검증되는 노드 수 : tensor(500)
테스트되는 노드 수 : tensor(1000)


### 3. Mini-batches

In [9]:
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

graph_dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
print(f"데이터세트 총 길이 : {len(graph_dataset)}","\n")

loader = DataLoader(graph_dataset, batch_size=32, shuffle=True)
print(f"배치 개수 :{len(graph_dataset)/32}","\n")

for idx,batch in enumerate(loader):
    print(f"{idx} : {batch}")
    print(f"배치안 그래프 개수 : {batch.num_graphs}","\n")
    break

데이터세트 총 길이 : 600 

배치 개수 :18.75 

0 : DataBatch(edge_index=[2, 4228], x=[1151, 21], y=[32], batch=[1151], ptr=[33])
배치안 그래프 개수 : 32 



### 4. Data Transforms

왜그런지 모르겠는데 데이터세트 다운 갑자기안됨;;; 나중에해야겠다.

In [13]:
# # 17000개의 3D shape point cloud가 담긴 데이터세트 (레이블은 16개의 클래스를 가짐)
# from torch_geometric.datasets import ShapeNet

# shapeNet_dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Motorbike'])

### 5. Learning Methods on Graphs

In [1]:
from torch_geometric.datasets import Planetoid

graph_dataset = Planetoid(root='/tmp/Cora', name="Cora")

In [7]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [8]:
# 그래프 속성 확인
print(f"전체 그래프 수 : {len(graph_dataset)}")
for idx,graph in enumerate(graph_dataset):
    print(f"====== {idx}번째 그래프의 특징은 다음과 같습니다. ======")
    print("그래프 KEY :",graph.keys)
    print("노드 수 :",graph.num_nodes) 
    print("엣지 수 :",graph.num_edges) 
    print("노드 특징 행렬 :",graph.num_node_features) 
    print("고립 노드 유무 :",graph.has_isolated_nodes())
    print("셀프 루프 유무 :",graph.has_self_loops()) 
    print("그래프 방향성 여부 :",graph.is_directed())

    print("======= 추가된 특징 =======")
    print("학습되는 노드 수 :", graph.train_mask.sum())
    print("검증되는 노드 수 :", graph.val_mask.sum())
    print("테스트되는 노드 수 :", graph.test_mask.sum())
    break
    
print(f"레이블 클래스 수 : {graph_dataset.num_classes}")

전체 그래프 수 : 1
====== 0번째 그래프의 특징은 다음과 같습니다. ======
그래프 KEY : ['y', 'train_mask', 'test_mask', 'edge_index', 'x', 'val_mask']
노드 수 : 2708
엣지 수 : 10556
노드 특징 행렬 : 1433
고립 노드 유무 : False
셀프 루프 유무 : False
그래프 방향성 여부 : False
======= 추가된 특징 =======
학습되는 노드 수 : tensor(140)
검증되는 노드 수 : tensor(500)
테스트되는 노드 수 : tensor(1000)
레이블 클래스 수 : 7


- GCN 이 어떤형태로 학습되는지 이해해야함
- GCNConv의 입출력이 어떤형태인지? 왜 conv1이랑 conv2에 입력이 두개인지 모르겠네 ...음... 같이들어가는건가?

In [10]:
class GCN(torch.nn.Module):
    def __init__(self,dataset):
        super(GCN,self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16,dataset.num_classes)
    
    def forward(self,data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x,edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x,edge_index)
        return F.log_softmax(x,dim=1)

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(graph_dataset).to(device)
data = graph_dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [14]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7990
